## Exercise 5 - Probabilistic models

First name: Brian
<br>
Last name: Schweigler
<br>
Matriculation number: 16-102-071


##### Q1: Represent each play by a vector with only the tf component. You can apply some preprocessing before generating this vector representation.
General imports and setup:

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import re
import numpy as np
import lxml.etree
import os
from scipy import stats
import nltk
import nltk.tokenize
import collections

np.random.seed(6)  # for reproducibility

PUNCT_RE = re.compile(r'[^\w\s]+$')


def is_punct(string):
    """Check if STRING is a punctuation marker or a sequence of
       punctuation markers.
    """
    return PUNCT_RE.match(string) is not None

def preprocess_text(text, language='French', lowercase=True):
    if lowercase:
        text = text.lower()
    if (language == 'French'):
        text = re.sub("-", " ", text)
        text = re.sub("l'", "le ", text)
        text = re.sub("d'", "de ", text)
        text = re.sub("c'", "ce ", text)
        text = re.sub("j'", "je ", text)
        text = re.sub("m'", "me ", text)
        text = re.sub("qu'", "que ", text)
        text = re.sub("'", " ' ", text)
        text = re.sub("quelqu'", "quelque ", text)
        text = re.sub("aujourd'hui", "aujourdhui", text)
    tokens = nltk.tokenize.word_tokenize(text, language=language)
    tokens = [token for token in tokens if not is_punct(token)]
    return tokens


subgenres = ('Comédie', 'Tragédie', 'Tragi-comédie')
plays, titles, genres = [], [], []
authors, years = [], []

for fn in os.scandir('theatre-classique'):
    # Only include XML files
    if not fn.name.endswith('.xml'):
        continue
    tree   = lxml.etree.parse(fn.path)
    genre  = tree.find('//genre')
    title  = tree.find('//title')
    author = tree.find('//author')
    year   = tree.find('//date')
    if genre is not None and genre.text in subgenres:
        lines = []
        for line in tree.xpath('//l|//p'):
            lines.append(' '.join(line.itertext()))
        text = '\n'.join(lines)
        plays.append(text)
        genres.append(genre.text)
        titles.append(title.text)
        authors.append(author.text)
        if year is not None:
            years.append(year.text)

print (len(plays), len(genres), len(titles), len(authors), len(years))



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
498 498 498 498 208


In [74]:
#nltk.download('punkt')
plays_tok = [preprocess_text(play, 'French') for play in plays]

def extract_vocabulary(tokenized_corpus, min_count=1, max_count=float('inf')):
    vocabulary = collections.Counter()
    for document in tokenized_corpus:
        vocabulary.update(document)
    vocabulary = {word for word, count in vocabulary.items()
                  if count >= min_count and count <= max_count}
    return sorted(vocabulary)

vocabulary = extract_vocabulary(plays_tok, min_count=2)
len(vocabulary)

38410

In [71]:
# Representation in a doc x term matrix for the French plays
def corpus2dtm(tokenized_corpus, vocabulary):
    "Transform a tokenized corpus into a document-term matrix"
    document_term_matrix = []
    for document in tokenized_corpus:
        document_counts = collections.Counter(document)
        row = [document_counts[word] for word in vocabulary]
        document_term_matrix.append(row)
    return document_term_matrix

# building the doc/term matrix in a few seconds for the French example
document_term_matrix = np.array(corpus2dtm(plays_tok, vocabulary))
print(f"document-term matrix with "
      f"|D| = {document_term_matrix.shape[0]} documents and "
      f"|V| = {document_term_matrix.shape[1]} words.")


document-term matrix with |D| = 498 documents and |V| = 62967 words.


Now we can calculate the tf component, which is the occurrence frequency of a term in the whole corpus. For this, we stem the words.

In [73]:
import operator
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
fr = SnowballStemmer('french')

stemmed_vocabulary = [fr.stem(word) for word in vocabulary]

counts = dict()

for word in stemmed_vocabulary:
    if word in counts:
        counts[word] += 1
    else:
        counts[word] = 1

tf_dict = counts
for key, value in counts.items():
    tf_dict[key] = value/len(stemmed_vocabulary)

tf_dict

{'.cour': 1.5881334667365444e-05,
 '.de': 1.5881334667365444e-05,
 '.du': 1.5881334667365444e-05,
 '.hé': 1.5881334667365444e-05,
 '.la': 1.5881334667365444e-05,
 '.le': 1.5881334667365444e-05,
 '.pein': 1.5881334667365444e-05,
 '.permet': 1.5881334667365444e-05,
 '.peut': 1.5881334667365444e-05,
 '.pispour': 1.5881334667365444e-05,
 '.que': 1.5881334667365444e-05,
 '.quel': 1.5881334667365444e-05,
 '.sais': 1.5881334667365444e-05,
 '.silenc': 1.5881334667365444e-05,
 '.tout': 1.5881334667365444e-05,
 '.trep': 1.5881334667365444e-05,
 '.violent': 1.5881334667365444e-05,
 '.vous': 1.5881334667365444e-05,
 '///rç': 1.5881334667365444e-05,
 '//catalogue.bnf.fr/ark': 1.5881334667365444e-05,
 '/12148/bpt6k286228': 1.5881334667365444e-05,
 '/musicien': 1.5881334667365444e-05,
 '01/06/1663': 1.5881334667365444e-05,
 '01/10/1650': 1.5881334667365444e-05,
 '02/12/1666': 1.5881334667365444e-05,
 '02/12/1671': 1.5881334667365444e-05,
 '04/02/1661': 1.5881334667365444e-05,
 '04/02/1670': 1.5881334

##### Q2: For each genre, it is possible to generate a “profile”, in the form of a single vector representing the entire set of plays corresponding to this genre. Build such a profile for each of the three genres (Comedy, Tragedy and Tragicomedy).



In [75]:
plays_com, titles_com, genres_com = [], [], []
authors_com, years_com = [], []

plays_tra, titles_tra, genres_tra = [], [], []
authors_tra, years_tra = [], []

plays_tra_com, titles_tra_com, genres_tra_com = [], [], []
authors_tra_com, years_tra_com = [], []
for fn in os.scandir('theatre-classique'):
    # Only include XML files
    if not fn.name.endswith('.xml'):
        continue
    tree = lxml.etree.parse(fn.path)
    genre = tree.find('//genre')
    title = tree.find('//title')
    author = tree.find('//author')
    year = tree.find('//date')
    if genre is not None and genre.text == 'Comédie':
        lines = []
        for line in tree.xpath('//l|//p'):
            lines.append(' '.join(line.itertext()))
        text = '\n'.join(lines)
        plays_com.append(text)
        genres_com.append(genre.text)
        titles_com.append(title.text)
        authors_com.append(author.text)
        if year is not None:
            years_com.append(year.text)
    elif genre is not None and genre.text == 'Tragédie':
        lines = []
        for line in tree.xpath('//l|//p'):
            lines.append(' '.join(line.itertext()))
        text = '\n'.join(lines)
        plays_tra.append(text)
        genres_tra.append(genre.text)
        titles_tra.append(title.text)
        authors_tra.append(author.text)
        if year is not None:
            years_tra.append(year.text)
    elif genre is not None and genre.text == 'Tragi-comédie':
        lines = []
        for line in tree.xpath('//l|//p'):
            lines.append(' '.join(line.itertext()))
        text = '\n'.join(lines)
        plays_tra_com.append(text)
        genres_tra_com.append(genre.text)
        titles_tra_com.append(title.text)
        authors_tra_com.append(author.text)
        if year is not None:
            years_tra_com.append(year.text)

plays_com_tok = [preprocess_text(play, 'French') for play in plays_com]
plays_tra_tok = [preprocess_text(play, 'French') for play in plays_tra]
plays_tra_com_tok = [preprocess_text(play, 'French') for play in plays_tra_com]
vocabulary_com = extract_vocabulary(plays_com_tok, min_count=2)
vocabulary_tra = extract_vocabulary(plays_tra_tok, min_count=2)
vocabulary_tra_com = extract_vocabulary(plays_tra_com_tok, min_count=2)

In [76]:
stemmed_vocabulary_com = [fr.stem(word) for word in vocabulary_com]
counts = dict()
for word in stemmed_vocabulary_com:
    if word in counts:
        counts[word] += 1
    else:
        counts[word] = 1

tf_dict_com = counts
for key, value in counts.items():
    tf_dict_com[key] = value / len(stemmed_vocabulary_com)
tf_dict_com

{'.de': 3.35795836131632e-05,
 '.que': 3.35795836131632e-05,
 '05/01/1667': 3.35795836131632e-05,
 '1': 3.35795836131632e-05,
 '1.': 3.35795836131632e-05,
 '10': 3.35795836131632e-05,
 '10.': 3.35795836131632e-05,
 '11': 3.35795836131632e-05,
 '11.': 3.35795836131632e-05,
 '12': 3.35795836131632e-05,
 '12.': 3.35795836131632e-05,
 '13': 3.35795836131632e-05,
 '13.': 3.35795836131632e-05,
 '14': 3.35795836131632e-05,
 '15': 3.35795836131632e-05,
 '1500': 3.35795836131632e-05,
 '16': 3.35795836131632e-05,
 '1635.': 3.35795836131632e-05,
 '1643': 3.35795836131632e-05,
 '1645': 3.35795836131632e-05,
 '1645.': 3.35795836131632e-05,
 '1646': 3.35795836131632e-05,
 '1649': 3.35795836131632e-05,
 '1649.': 3.35795836131632e-05,
 '1650': 3.35795836131632e-05,
 '1653': 3.35795836131632e-05,
 '1653.': 3.35795836131632e-05,
 '1654': 3.35795836131632e-05,
 '1654.': 3.35795836131632e-05,
 '1656.': 3.35795836131632e-05,
 '1658': 3.35795836131632e-05,
 '1658.': 3.35795836131632e-05,
 '1660': 3.35795836

In [77]:
stemmed_vocabulary_tra = [fr.stem(word) for word in vocabulary_tra]
counts = dict()
for word in stemmed_vocabulary_tra:
    if word in counts:
        counts[word] += 1
    else:
        counts[word] = 1

tf_dict_tra = counts
for key, value in counts.items():
    tf_dict_tra[key] = value / len(stemmed_vocabulary_tra)
tf_dict_tra

{'1': 4.93339911198816e-05,
 '1.': 4.93339911198816e-05,
 '10': 4.93339911198816e-05,
 '10.': 4.93339911198816e-05,
 '106': 4.93339911198816e-05,
 '11': 4.93339911198816e-05,
 '11.': 4.93339911198816e-05,
 '12': 4.93339911198816e-05,
 '12.': 4.93339911198816e-05,
 '13': 4.93339911198816e-05,
 '13.': 4.93339911198816e-05,
 '14': 4.93339911198816e-05,
 '14.': 4.93339911198816e-05,
 '15': 4.93339911198816e-05,
 '1575': 4.93339911198816e-05,
 '16': 4.93339911198816e-05,
 '16.': 4.93339911198816e-05,
 '1617': 4.93339911198816e-05,
 '1635': 4.93339911198816e-05,
 '1636': 4.93339911198816e-05,
 '1636.': 4.93339911198816e-05,
 '1644.': 4.93339911198816e-05,
 '1645.': 4.93339911198816e-05,
 '1647': 4.93339911198816e-05,
 '1648.': 4.93339911198816e-05,
 '1653': 4.93339911198816e-05,
 '1653.': 4.93339911198816e-05,
 '1654': 4.93339911198816e-05,
 '1663.': 4.93339911198816e-05,
 '1664': 4.93339911198816e-05,
 '1665': 4.93339911198816e-05,
 '1665.': 4.93339911198816e-05,
 '1666': 4.93339911198816e-

In [78]:
stemmed_vocabulary_tra_com = [fr.stem(word) for word in vocabulary_tra_com]
counts = dict()
for word in stemmed_vocabulary_tra_com:
    if word in counts:
        counts[word] += 1
    else:
        counts[word] = 1

tf_dict_tra_com = counts
for key, value in counts.items():
    tf_dict_tra_com[key] = value / len(stemmed_vocabulary_tra_com)
tf_dict_tra_com

{'10.': 9.19793966151582e-05,
 '1637.': 9.19793966151582e-05,
 '1639.': 9.19793966151582e-05,
 '1642.': 9.19793966151582e-05,
 '1646.': 9.19793966151582e-05,
 '1660.': 9.19793966151582e-05,
 '1662.': 9.19793966151582e-05,
 '18.': 9.19793966151582e-05,
 '19.': 9.19793966151582e-05,
 '2.': 9.19793966151582e-05,
 '2006': 9.19793966151582e-05,
 '2008': 9.19793966151582e-05,
 '2010': 9.19793966151582e-05,
 '2012': 9.19793966151582e-05,
 '2015': 9.19793966151582e-05,
 '28.': 9.19793966151582e-05,
 '7': 9.19793966151582e-05,
 '7.': 9.19793966151582e-05,
 '8.': 9.19793966151582e-05,
 'a': 9.19793966151582e-05,
 'a.': 9.19793966151582e-05,
 'abaiss': 0.0005518763796909492,
 'abaissement': 9.19793966151582e-05,
 'abandon': 0.000919793966151582,
 'abandonnent': 9.19793966151582e-05,
 'abat': 9.19793966151582e-05,
 'abattr': 9.19793966151582e-05,
 'abattu': 0.0001839587932303164,
 'abattus': 9.19793966151582e-05,
 'abhorr': 9.19793966151582e-05,
 'abject': 0.0001839587932303164,
 'abois': 9.197939

##### Q3: How many terms with a weight strictly larger than 0 do you have in each text genre profile?

This is simply the lengths of the tf vectors (note that we stemmed the terms!).

In [81]:
print("Terms for comedy: ", len(tf_dict_com))
print("Terms for tragedy: ", len(tf_dict_tra))
print("Terms for tragi-comedy: ", len(tf_dict_tra_com))

Terms for comedy:  14504
Terms for tragedy:  9227
Terms for tragi-comedy:  5327


##### Q4: Select randomly 10 plays for each text genre. Represent each play by a vector.

In [9]:
# TODO

##### Q5: For each text genre and play, how many terms with a weight strictly larger than 0 do you have in the vector?

This is again simply the lengths of the tf vectors.

In [9]:
# TODO
# print("Terms for comedy: ", len(tf_dict_com))
# print("Terms for tragedy: ", len(tf_dict_tra))
# print("Terms for tragi-comedy: ", len(tf_dict_tra_com))

##### Q6: For each text genre and play, how many terms with a weight strictly equal to 1 do you have in the vector?

With tf, none of them will have a weight of 1, but we can look at the max values:


In [82]:
# TODO Max of dict with key
#max(stats.items(), key=operator.itemgetter(1))[0]
#max(stats.values())